### Setting Up External Authentication
In this exercise, you set up server1 for external authentication on the labipa.example.com server. This exercise assumes you are using the labipa server that is described in Appendix D, “Setting Up Identity Management,” or that you are using the lab virtual machines (VMs) available for download at http://www.rhatcert.com. All steps in this exercise are performed on server1.
1. Verify that you can reach the labipa.example.com server by name, by using ping labipa.example.com. If this does not work, fix before proceeding by either including an entry in DNS that resolves labipa.example.com to its hostname or by using the DNS server that is available through the labipa server.
2. Type ```yum groups install “Directory Client” -y``` (you can use `yum group list`, or `yum group info '*' | less +/sssd`)
3. Type mkdir /etc/openldap/cacerts.
4. Copy the certificate from the IPA server to your local server by using scp labipa.example.com:/etc/ipa/ca.crt /etc/openldap/cacerts.
5. Type vim /etc/sysconfig/authconfig. Make sure that this file contains the following configuration options:
```
USESSSDAUTH=yes
USESSSD=yes
FORCELEGACY=no
```
6. As root, start authconfig-tui. In User Information, select Cache Information, Use LDAP, and under Authentication, select Use LDAP Authentication.
7. In the LDAP Settings screen, select Use TLS and specify the following:
```
Server:   labipa.example.com
Base DN: dc=example,dc=com
```
8. Complete the setup procedure in authconfig-tui. Once the configuration is written to your system, use `cat /etc/sssd/sssd.conf` to verify the configuration has been committed to your server.
9. Type `systemctl restart sssd` to restart the sssd service.
10. From a shell, type `su - ldapuser1`. You should now see the message “cannot change to /home/lisa: No such file or directory,” which is fine because we have not configured any home directory automounting.
11. Type `id`. This will show that you are currently logged in as user ldapuser1.

### Configuring Kerberos Authorization

1. On server1, open a root shell and start `authconfig-tui`.
2. Type `yum install -y pam_krb5 krb5-workstation` to install the RPM packages that add Kerberos support to all
applications that support it.
    IMPORTANT: use `yum groups install “Directory Client"` to install krb5, you can also check what is missing using: `authconfig --test`


3. On the authentication Configuration screen, under Authentication, select Use Kerberos to enable Kerberos authorization.
4. On the LDAP Settings screen, do not change anything; you configured this in the previous exercise.
5. In the Kerberos Settings screen, enter the following parameters:

```
Realm:    EXAMPLE.COM
KDC: labipa.example.com
Admin Server: labipa.example.com
```

6. When you are back on the shell prompt, type `kinit admin` as root. When prompted, provide the password of the admin user on the labipa server. (Typically set to password or password123). This verifies that the host can authenticate and shows that Kerberos authentication configuration is correct.
7. Now that you know that the host can authenticate, type kinit ldapuser1. Enter the password for user ldapuser1. This proves that Kerberos authentication works correctly.

### Making an iSCSI Connection
1. On server 2, open a root shell and type `yum -y install iscsi-initiator-utils lsscsi` to install the software that you need to perform this exercise.
2. Type `iscsiadm --mode discovery --type sendtargets --portal 192.168.4.210 --discover`. This should return the name of the iSCSI target as you have configured it in the previous exercise.
3. Next, type `iscsiadm --mode node --targetname iqn.2014-04.com.example:target --portal 192.168.4.210 --login`.
4. The iSCSI devices should now be available. Type `lsscsi` to show them. You should see three LIO devices. Use `iscsiadm -m session -P3`, which also shows all disks.
5. On the first iSCSI device (I’ll assume that it is /dev/sdb in this exercise, but it can be a different device on your server depending on the configuration that is used), type `mkfs.xfs /dev/sdb`.
6. Use `blkid /dev/sdb` to get the UUID that is set for the XFS file system that you have just created on /dev/sdb.
7. Create a mount point for the iSCSI disk, using `mkdir /mnt/iscsi`.
8. Type `vim /etc/fstab` to open the /etc/fstab file in an editor and add a line that looks like the following. (Make sure to replace the UUID with the UUID you have found in step 6 of this exercise.)
`UUID=XXXXXXXX-XXXX-XXXX-XXXXXX /mnt/iscsi       xfs _netdev   0 2`

9. Type `mount -a`. This should mount the iSCSI disk. Type mount without any arguments to verify.

### Setting Up Remote Logging
In this exercise, you configure server2 to receive messages from remote rsyslogd processes. You define the rsyslogd configuration on server1 to forward messages to server2 and open a firewall port on server2 that allows for log file message reception.
- Open a root shell on server2. Then, open the configuration file /etc/rsyslog.conf.
- In rsyslog.conf enable the following two lines to enable log reception on TCP port 514:

```
$ModLoad imtcp
$InputTCPServerRun 514
```


- Close the configuration file and type systemctl restart rsyslog.service to restart the rsyslogd service. This allows the rsyslogd process on the log server to receive messages from others.

`systemctl list-unit-files --all | grep enable | grep rsyslog`

- Still on server2, open the firewall to accept messages on TCP port 514, using the following two lines:

```
firewall-cmd --add-port=514/tcp
firewall-cmd --add-port=514/tcp --permanent
```

- Open a root shell on server1, and scroll down to the end of the configuration file. Here, you find the following example configuration line:

```
#*.* @@remote-host:514
```

This line shows how to configure your server to forward messages to a remote server. Change this line to read like the following to forward messages to rsyslogd on server2:
Click here to view code image

```
*.*       @@server2.example.com:514
```

- Use `systemctl restart rsyslog` to restart the rsyslogd process and start logging to the remote server.
- On server1 use logger command to test remote logging:

`logger test`

- On server2 do: `tail -f /var/log/messages` and you shoud see log entry like:
`Dec 9 18:21:05 server1 user: test`

### Configuring Port Forwarding

To complete this exercise, you need two servers. Port forwarding is configured on server1, and server2 is used for testing purposes. Do not make permanent configurations; they might interfere with tasks you need to accomplish in exercises in upcoming chapters.

1. On server1, type `firewall-cmd --add-forward-port=port=4404:proto=tcp:toport=22`.
2. From server2, use `ssh -p 4044 server1` to verify that the ssh service can now be reached on port 4044 also. Note that because this is a same-host configuration, no additional masquerading rules are needed.
